## The parser class ##

In [86]:


from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


list_of_prefix = """
PREFIX prop: <http://dbpedia.org/property/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX db: <http://dbpedia.org/>
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX onto: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dbpedia2: <http://dbpedia.org/property/>
"""

# will change it to be received from the user
input_query = """
SELECT ?place
WHERE {
   dbr:Albert_Einstein dbo:birthPlace ?place .
   ?subject ?predicate ?object .
}
"""


parsed_query = SPARQL(input_query)





class Triple:  # chaneg name
    def __init__(self, sub, pre, obj):
        self.sub = sub
        self.pre = pre
        self.obj = obj

    def get_subject(self):

        return self.sub

    def get_predicate(self):
        return self.pre

    def get_object(self):
        return self.obj


triples_to_query = []

for s in parsed_query.triple_list:
    triples_to_query.append(
        Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))
    
original_query_t = [[],[],[]]

for x in triples_to_query:
    original_query_t[0].append(str(x.get_subject()))
    original_query_t[1].append(str(x.get_predicate()))
    original_query_t[2].append(str(x.get_object()))

# for T in t:
#   for z in T :
#      print((z))


## The approx function call 

In [98]:


# STARTING POINT APPROX -> GET APPROX -> CALCULATE -> EXECUTE_Q -> BUILD_QUERY -> 

#  Goes to the dictionary and creates a new index which is the cost and calls the approx queries to calculate the values in that cost 
def approx(cost):
    for a in range (cost+1):
        approx_queries[str(a)] = get_approx(a)


#  this gets the index(cost) and returns a whole list of triples list for that cost 
# the output is the final list
def get_approx(a):
    value = []
    if a == 0:
        value.append(triples_to_query)
        # value = [value]
        execute_q(value,a)
    else:
        pre_approx = approx_queries[str(a-1)] # here we get the list of triples list belonging to the previous query
        for triple in pre_approx: # here we iterate over the list of triples list
            new_approx =  calculate(triple, a) # here creating the temp list of triples by calling the calculate function(A single list of triples , cost)
            execute_q(new_approx,a)
            value += new_approx # here we append the list belonging to the previous query to the total list
    return value

approx_queries = dict()
# approx(0)



# for key in approx_queries.keys():
#     print (f" ******* key = {key} *******")
#     for b in approx_queries[str(key)]: 
#         # print(len(b))  # b is  a list of triples
#         for c in b: # c is a list triple
#             print((str(c.get_subject()) + " " + str(c.get_predicate()) + " " + str(c.get_object())))
#         print ("-----------------------")
       


## calculation the approx 

In [95]:
from collections import deque

def calculate(triples,cost):
    values = []
    for triple in triples:
        index = triples.index(triple)

        if(triple.get_subject() in original_query_t[0]):
            values.append(left_insertion(triples, cost,index))
        # if(triple.get_predicate() in original_query_t[1]): 
        #     values.append(sub(triples, cost, index))
        if(triple.get_object() in original_query_t[2]):
             values.append(right_insertion(triples, cost,index))
    return values
    

def left_insertion(triple,cost,index):
    list_of_approx = triple[:]
    list_of_approx.insert(index,(Triple(list_of_approx[index].get_subject(),f"?p{cost}",f"?o{cost}")))
    list_of_approx[index +1] = Triple(f"?o{cost}",list_of_approx[index+1].get_predicate(),list_of_approx[index+1].get_object())
    return list_of_approx

def right_insertion(triple,cost,index):

    list_of_approx = triple[:]
    list_of_approx.insert(index+1,(Triple(f"?o{cost}",f"?p{cost}",list_of_approx[index].get_object())))
    list_of_approx[index] = Triple(list_of_approx[index].get_subject(),list_of_approx[index].get_predicate(),f"?o{cost}")
    return list_of_approx

def sub(triples,cost,index):
    list_of_approx = triples[:]
    list_of_approx[index] = (Triple(triples[index].get_subject(),f"?p{cost}",triples[index].get_object()))
    return list_of_approx






# x = left_insertion([Triple("albert","birth","?o1"),Triple("?o1","?p1","?place")],1)
# y = right_insertion(x,2)
# z = sub

# for triple in y:
#     print(triple.get_subject())
#     print(triple.get_predicate())
#     print(triple.get_object())
#     print("*************")
        

## The query builder ##
    

In [94]:

def execute_q(queries,cost):
   for query in queries:
        run_query(build_query(query),cost)
        # call  the endpoint to get shit 

def build_query(queries):
    sparql_query = f"{parsed_query.former} where {{\n"
    for triple in queries:
        sparql_query += f"{triple.get_subject()} {triple.get_predicate()} {triple.get_object()} .\n"
    sparql_query += "}\n LIMIT 10"


    return list_of_prefix + sparql_query


# print(get_query())


# for t in triples_to_query:
#     print("sub " + t.get_subject())
#     print("pre " + t.get_predicate())
#     print("obj " + t.get_object())
#     print('------------------')

# get_query()

## The endpoint ## 

In [101]:

def run_query(query_to_execute,cost): 
    sparql = SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
    sparql.setQuery(query_to_execute)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()  
    
    index = results['head']['vars']
    
    result_list = []
    ind_reslut = []
    
    
    for y in index:
        for x1 in results['results']['bindings']:
            ind_reslut.append(x1[y]['value'])
        result_list.append(ind_reslut)
        ind_reslut = []
        
    wr_my_list(result_list,cost)



In [109]:
def wr_my_list(l1list,cost):
    file_path = 'test.txt'
    with open(file_path, 'a') as file:
        
        for l1 in zip(*l1list):
            for l2 in l1:
                file.write(str(l2) + " " + str(cost) + "\n" )
            # file.write('\n ------------------ \n')


In [110]:
approx(5)

URLError: <urlopen error [Errno 113] No route to host>